<a href="https://colab.research.google.com/github/yoonkim313/dataCampusProject-Team10/blob/master/ocr/test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This can be executed in https://colab.research.google.com "Python 3 / GPU" runtime.

In [ ]:
!git clone https://github.com/yoonkim313/dataCampusProject-Team10.git

모델 저장 경로 확인

In [ ]:
% cd /content/dataCampusProject-Team10/ocr/saved_models

In [ ]:
models = {
    'best_accuracy.pth': 'https://drive.google.com/file/d/1JqMF8-FSgpAaSfhq-CFbERXpX5rTTQrW/view?usp=sharing'}
for k, v in models.items():
    doc_id = v[v.find('=')+1:]
    !curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=$doc_id" > /tmp/intermezzo.html
    !curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > $k

!ls -al *.pth

ocr 디렉토리로 이동

In [ ]:
% cd /content/dataCampusProject-Team10/ocr/

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python3 test.py \
--eval_data /content/drive/'Shared drives'/'BigDATA TEAM 10'/ocr/data/ \
--benchmark_all_eval \
--Transformation TPS \
--FeatureExtraction VGG \
--SequenceModeling BiLSTM \
--Prediction Attn \
--saved_model /content/drive/'Shared drives'/'BigDATA TEAM 10'/ocr/model/best_accuracy.pth

In [ ]:
output = !CUDA_VISIBLE_DEVICES=0 python3 demo.py \
--Transformation TPS \
--FeatureExtraction VGG \
--SequenceModeling BiLSTM \
--Prediction Attn \
--image_folder test_image \
--saved_model /content/drive/'Shared drives'/'BigDATA TEAM 10'/ocr/model/best_accuracy.pth

In [ ]:
from IPython.core.display import display, HTML
from PIL import Image
import base64
import io
import pandas as pd

data = pd.DataFrame()
for ind, row in enumerate(output[output.index('image_path \t predicted_labels \t confidence score')+2:]):
    row = row.split('\t')
    filename = row[0].strip()
    label = row[1].strip()
    conf = row[2].strip()
    img = Image.open(filename)
    img_buffer = io.BytesIO()
    img.save(img_buffer, format="PNG")
    imgStr = base64.b64encode(img_buffer.getvalue()).decode("utf-8")
    
    data.loc[ind, 'img'] = '<img src="data:image/png;base64,{0:s}">'.format(imgStr)
    data.loc[ind, 'id'] = filename
    data.loc[ind, 'label'] = label
    data.loc[ind, 'conf'] = conf
    
html_all = data.to_html(escape=False)
display(HTML(html_all))